### Reading In The Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
import matplotlib.pyplot as plt

In [4]:
import seaborn as sns

In [12]:
pd.set_option('max_column', None)
data = pd.read_csv('Week1_challenge_data_source.csv')
df = data
df.head()

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),DL TP < 50 Kbps (%),50 Kbps < DL TP < 250 Kbps (%),250 Kbps < DL TP < 1 Mbps (%),DL TP > 1 Mbps (%),UL TP < 10 Kbps (%),10 Kbps < UL TP < 50 Kbps (%),50 Kbps < UL TP < 300 Kbps (%),UL TP > 300 Kbps (%),HTTP DL (Bytes),HTTP UL (Bytes),Activity Duration DL (ms),Activity Duration UL (ms),Dur. (ms).1,Handset Manufacturer,Handset Type,Nb of sec with 125000B < Vol DL,Nb of sec with 1250B < Vol UL < 6250B,Nb of sec with 31250B < Vol DL < 125000B,Nb of sec with 37500B < Vol UL,Nb of sec with 6250B < Vol DL < 31250B,Nb of sec with 6250B < Vol UL < 37500B,Nb of sec with Vol DL < 6250B,Nb of sec with Vol UL < 1250B,Social Media DL (Bytes),Social Media UL (Bytes),Google DL (Bytes),Google UL (Bytes),Email DL (Bytes),Email UL (Bytes),Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,1.311448e+19,4/4/2019 12:01,770.0,4/25/2019 14:35,662.0,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9.16456699548519E+015,42.0,5.0,23.0,44.0,NaN,NaN,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,NaN,NaN,37624.0,38787.0,1.823653e+09,Samsung,Samsung Galaxy A5 Sm-A520F,NaN,NaN,NaN,NaN,NaN,NaN,213.0,214.0,1545765.0,24420.0,1634479.0,1271433.0,3563542.0,137762.0,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0
1,1.311448e+19,4/9/2019 13:04,235.0,4/25/2019 8:15,606.0,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,65.0,5.0,16.0,26.0,NaN,NaN,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,NaN,NaN,168.0,3560.0,1.365104e+09,Samsung,Samsung Galaxy J5 (Sm-J530),NaN,NaN,NaN,NaN,NaN,NaN,971.0,1022.0,1926113.0,7165.0,3493924.0,920172.0,629046.0,308339.0,20247395.0,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0
2,1.311448e+19,4/9/2019 17:42,1.0,4/25/2019 11:58,652.0,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,NaN,NaN,6.0,9.0,NaN,NaN,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,1.361763e+09,Samsung,Samsung Galaxy A8 (2018),NaN,NaN,NaN,NaN,NaN,NaN,751.0,695.0,1684053.0,42224.0,8535055.0,1694064.0,2690151.0,672973.0,19725661.0,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0
3,1.311448e+19,4/10/2019 0:31,486.0,4/25/2019 7:36,171.0,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,NaN,NaN,44.0,44.0,NaN,NaN,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,NaN,NaN,3330.0,37882.0,1.321510e+09,undefined,undefined,NaN,NaN,NaN,NaN,NaN,NaN,17.0,207.0,644121.0,13372.0,9023734.0,2788027.0,1439754.0,631229.0,21388122.0,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0
4,1.311448e+19,4/12/2019 20:10,565.0,4/25/2019 10:40,954.0,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,NaN,NaN,6.0,9.0,NaN,NaN,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,1.089009e+09,Samsung,Samsung Sm-G390F,NaN,NaN,NaN,NaN,NaN,NaN,607.0,604.0,862600.0,50188.0,6248284.0,1500559.0,1936496.0,173853.0,15259380.0,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0


In [10]:
# Reading in and printing out description of all columns in the dataset
field_desc = pd.read_excel('Field_Descriptions.xlsx')
count = 0
for i in field_desc['Fields']:
    print(f'{i}: {field_desc.Description[count]}\n')
    count += 1

bearer id: xDr session identifier

Dur. (ms): Total Duration of the xDR (in ms)

Start: Start time of the xDR (first frame timestamp)

Start ms: Milliseconds offset of start time for the xDR (first frame timestamp)

End: End time of the xDR (last frame timestamp)

End ms: Milliseconds offset of end time of the xDR (last frame timestamp)

Dur. (s): Total Duration of the xDR (in s)

IMSI: International Mobile Subscriber Identity

MSISDN/Number: MS International PSTN/ISDN Number of mobile - customer number

IMEI: International Mobile Equipment Identity

Last Location Name: User location call name (2G/3G/4G) at the end of the bearer

Avg RTT DL (ms): Average Round Trip Time measurement Downlink direction (msecond)

Avg RTT UL (ms): Average Round Trip Time measurement Uplink direction (msecond)

Avg Bearer TP DL (kbps): Average Bearer Throughput for Downlink (kbps) - based on BDR duration

Avg Bearer TP UL (kbps): Average Bearer Throughput for uplink (kbps) - based on BDR duration

TCP DL R

In [13]:
# Checking the shape of the dataset
data.shape

(150001, 55)

The dataset has 150001 rows and 55 columns

### Performing Pre-processing

In [15]:
# function to check all columns for percentage missing values
# And return list of columns with 30% or more of values missing
def missing_vals(df):
    count = 0
    drop = []
    for i in df.columns:
        print(f'The column "{i}" has {round(df.isnull().sum()[count]/df[i].shape[0] * 100, 2)}% of values missing.\n')
        if round(df.isnull().sum()[count]/df[i].shape[0] * 100, 2) >= 30:
            drop.append(i)
        count+=1
    return drop

In [16]:
# using the missing vals function and storing the list in a variable
drop = missing_vals(df)

The column "Bearer Id" has 0.66% of values missing.

The column "Start" has 0.0% of values missing.

The column "Start ms" has 0.0% of values missing.

The column "End" has 0.0% of values missing.

The column "End ms" has 0.0% of values missing.

The column "Dur. (ms)" has 0.0% of values missing.

The column "IMSI" has 0.38% of values missing.

The column "MSISDN/Number" has 0.71% of values missing.

The column "IMEI" has 0.38% of values missing.

The column "Last Location Name" has 0.77% of values missing.

The column "Avg RTT DL (ms)" has 18.55% of values missing.

The column "Avg RTT UL (ms)" has 18.54% of values missing.

The column "Avg Bearer TP DL (kbps)" has 0.0% of values missing.

The column "Avg Bearer TP UL (kbps)" has 0.0% of values missing.

The column "TCP DL Retrans. Vol (Bytes)" has 58.76% of values missing.

The column "TCP UL Retrans. Vol (Bytes)" has 64.43% of values missing.

The column "DL TP < 50 Kbps (%)" has 0.5% of values missing.

The column "50 Kbps < DL TP 

In [17]:
# checking columns that will be dropped
drop

['TCP DL Retrans. Vol (Bytes)',
 'TCP UL Retrans. Vol (Bytes)',
 'HTTP DL (Bytes)',
 'HTTP UL (Bytes)',
 'Nb of sec with 125000B < Vol DL',
 'Nb of sec with 1250B < Vol UL < 6250B',
 'Nb of sec with 31250B < Vol DL < 125000B',
 'Nb of sec with 37500B < Vol UL',
 'Nb of sec with 6250B < Vol DL < 31250B',
 'Nb of sec with 6250B < Vol UL < 37500B']

In [18]:
# dropping columns with over 30% of data missing and checking shape
df.drop(drop, axis=1, inplace=True)
df.shape

(150001, 45)

In [19]:
# checking head of dataframe
df.head()

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),DL TP < 50 Kbps (%),50 Kbps < DL TP < 250 Kbps (%),250 Kbps < DL TP < 1 Mbps (%),DL TP > 1 Mbps (%),UL TP < 10 Kbps (%),10 Kbps < UL TP < 50 Kbps (%),50 Kbps < UL TP < 300 Kbps (%),UL TP > 300 Kbps (%),Activity Duration DL (ms),Activity Duration UL (ms),Dur. (ms).1,Handset Manufacturer,Handset Type,Nb of sec with Vol DL < 6250B,Nb of sec with Vol UL < 1250B,Social Media DL (Bytes),Social Media UL (Bytes),Google DL (Bytes),Google UL (Bytes),Email DL (Bytes),Email UL (Bytes),Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,1.311448e+19,4/4/2019 12:01,770.0,4/25/2019 14:35,662.0,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9.16456699548519E+015,42.0,5.0,23.0,44.0,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,37624.0,38787.0,1.823653e+09,Samsung,Samsung Galaxy A5 Sm-A520F,213.0,214.0,1545765.0,24420.0,1634479.0,1271433.0,3563542.0,137762.0,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0
1,1.311448e+19,4/9/2019 13:04,235.0,4/25/2019 8:15,606.0,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,65.0,5.0,16.0,26.0,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,168.0,3560.0,1.365104e+09,Samsung,Samsung Galaxy J5 (Sm-J530),971.0,1022.0,1926113.0,7165.0,3493924.0,920172.0,629046.0,308339.0,20247395.0,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0
2,1.311448e+19,4/9/2019 17:42,1.0,4/25/2019 11:58,652.0,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,NaN,NaN,6.0,9.0,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,1.361763e+09,Samsung,Samsung Galaxy A8 (2018),751.0,695.0,1684053.0,42224.0,8535055.0,1694064.0,2690151.0,672973.0,19725661.0,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0
3,1.311448e+19,4/10/2019 0:31,486.0,4/25/2019 7:36,171.0,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,NaN,NaN,44.0,44.0,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,3330.0,37882.0,1.321510e+09,undefined,undefined,17.0,207.0,644121.0,13372.0,9023734.0,2788027.0,1439754.0,631229.0,21388122.0,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0
4,1.311448e+19,4/12/2019 20:10,565.0,4/25/2019 10:40,954.0,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,NaN,NaN,6.0,9.0,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,1.089009e+09,Samsung,Samsung Sm-G390F,607.0,604.0,862600.0,50188.0,6248284.0,1500559.0,1936496.0,173853.0,15259380.0,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 45 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Bearer Id                       149010 non-null  float64
 1   Start                           150000 non-null  object 
 2   Start ms                        150000 non-null  float64
 3   End                             150000 non-null  object 
 4   End ms                          150000 non-null  float64
 5   Dur. (ms)                       150000 non-null  float64
 6   IMSI                            149431 non-null  float64
 7   MSISDN/Number                   148935 non-null  float64
 8   IMEI                            149429 non-null  float64
 9   Last Location Name              148848 non-null  object 
 10  Avg RTT DL (ms)                 122172 non-null  float64
 11  Avg RTT UL (ms)                 122189 non-null  float64
 12  Avg Bearer TP DL

In [21]:
# Function to determine outlier using interquartile range
def outlier_det(df, col):

    return (df.describe()[col][6] - df.describe()[col][4]) * 1.5 + df.describe()[col][6]


# Function to check specified columns for outliers and store row numbers
def outlier_dict(df):

    drop_rows = {}
    for i in df.columns:
        test = outlier_det(df, i)
        count = 0
        lst = []
        print(f'The numer of ouliers in the column {i} is:\n')
        for j in df[i]:
            if j >= test:
                lst.append(count)
                drop_rows[count] = count
                
            else: continue
            count +=1
        print(f'{len(lst)}\n')

    return drop_rows

In [22]:
# creating list of columns to check for outliers
cols = []
cols.extend(list(df.columns[12:14])) 
cols.extend(list(df.columns[27:]))
cols

['Avg Bearer TP DL (kbps)',
 'Avg Bearer TP UL (kbps)',
 'Nb of sec with Vol DL < 6250B',
 'Nb of sec with Vol UL < 1250B',
 'Social Media DL (Bytes)',
 'Social Media UL (Bytes)',
 'Google DL (Bytes)',
 'Google UL (Bytes)',
 'Email DL (Bytes)',
 'Email UL (Bytes)',
 'Youtube DL (Bytes)',
 'Youtube UL (Bytes)',
 'Netflix DL (Bytes)',
 'Netflix UL (Bytes)',
 'Gaming DL (Bytes)',
 'Gaming UL (Bytes)',
 'Other DL (Bytes)',
 'Other UL (Bytes)',
 'Total UL (Bytes)',
 'Total DL (Bytes)']

In [23]:
# checking for outliers in selected columns and the number rows with ouliers
outliers = outlier_dict(df[cols])

The numer of ouliers in the column Avg Bearer TP DL (kbps) is:

13234

The numer of ouliers in the column Avg Bearer TP UL (kbps) is:

21531

The numer of ouliers in the column Nb of sec with Vol DL < 6250B is:

24544

The numer of ouliers in the column Nb of sec with Vol UL < 1250B is:

25154

The numer of ouliers in the column Social Media DL (Bytes) is:

0

The numer of ouliers in the column Social Media UL (Bytes) is:

0

The numer of ouliers in the column Google DL (Bytes) is:

0

The numer of ouliers in the column Google UL (Bytes) is:

0

The numer of ouliers in the column Email DL (Bytes) is:

0

The numer of ouliers in the column Email UL (Bytes) is:

0

The numer of ouliers in the column Youtube DL (Bytes) is:

0

The numer of ouliers in the column Youtube UL (Bytes) is:

0

The numer of ouliers in the column Netflix DL (Bytes) is:

0

The numer of ouliers in the column Netflix UL (Bytes) is:

0

The numer of ouliers in the column Gaming DL (Bytes) is:

0

The numer of oulier

In [24]:
# dropping rows with outliers and checking shape
df.drop(list(outliers.keys()), axis=0, inplace=True)
df.shape

(124847, 45)

In [35]:
# function to forward fill missing column values then backward fill remaining missing values
def ffill_bfill_cols(df, col):
    df[col] = df[col].fillna(method='ffill')
    df[col] = df[col].fillna(method='bfill')


    return df[col]

# function to fill missing values using mode
def mode_fill(df, col):
    df[col] = df[col].fillna(df[col].mode()[0])

    return df[col]

# function to convert column to datetime format
def datetime(df, col):
    df[col] = pd.to_datetime(df[col], infer_datetime_format=True)
    
    return df[col]

# function to fill missing values based on column types
def fill_cols(df):
    for i in df.columns:
        if df.dtypes[i] == 'float64':
            df[i] = ffill_bfill_cols(df, i)
        elif df.dtypes[i] == 'O':
            df[i] = mode_fill(df, i)
        else: continue
    
    return df

In [32]:
# creating new dataframe with missing values filled
df_pp = fill_cols(df)

In [33]:
# checking to make sure there are no issing values anymore
df_pp.isna().sum()

Bearer Id                         0
Start                             0
Start ms                          0
End                               0
End ms                            0
Dur. (ms)                         0
IMSI                              0
MSISDN/Number                     0
IMEI                              0
Last Location Name                0
Avg RTT DL (ms)                   0
Avg RTT UL (ms)                   0
Avg Bearer TP DL (kbps)           0
Avg Bearer TP UL (kbps)           0
DL TP < 50 Kbps (%)               0
50 Kbps < DL TP < 250 Kbps (%)    0
250 Kbps < DL TP < 1 Mbps (%)     0
DL TP > 1 Mbps (%)                0
UL TP < 10 Kbps (%)               0
10 Kbps < UL TP < 50 Kbps (%)     0
50 Kbps < UL TP < 300 Kbps (%)    0
UL TP > 300 Kbps (%)              0
Activity Duration DL (ms)         0
Activity Duration UL (ms)         0
Dur. (ms).1                       0
Handset Manufacturer              0
Handset Type                      0
Nb of sec with Vol DL < 6250

In [37]:
# changing start and end columns to datetime type
for i in ['Start','End']:
    
    df_pp[i] = datetime(df_pp, i)
    
df_pp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124847 entries, 25154 to 150000
Data columns (total 45 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   Bearer Id                       124847 non-null  float64       
 1   Start                           124847 non-null  datetime64[ns]
 2   Start ms                        124847 non-null  float64       
 3   End                             124847 non-null  datetime64[ns]
 4   End ms                          124847 non-null  float64       
 5   Dur. (ms)                       124847 non-null  float64       
 6   IMSI                            124847 non-null  float64       
 7   MSISDN/Number                   124847 non-null  float64       
 8   IMEI                            124847 non-null  float64       
 9   Last Location Name              124847 non-null  object        
 10  Avg RTT DL (ms)                 124847 non-null  flo